In [1]:
import sys
sys.path.insert(0,"/home2/tathagato/summarization/MACSUM/naacl")
import os
from eval import evaluate
import pickle as pkl

[nltk_data] Downloading package punkt to /home2/tathagato/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home2/tathagato/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home2/tathagato/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
all_result_paths = [
    "/scratch/tathagato/openai_outputs/all_results.pkl",
    "/scratch/tathagato/openai_outputs/all_results_rest.pkl"
]
# read the two pickle files and merge them and save as final_results.pkl
final_results = {}
for result_path in all_result_paths:
    with open(result_path, "rb") as f:
        results = pkl.load(f)
        final_results.update(results)
with open("/scratch/tathagato/openai_outputs/final_results.pkl", "wb") as f:
    pkl.dump(final_results, f)
print(len(final_results))

37


In [4]:
print(final_results.keys())

dict_keys(['/scratch/tathagato/naacl/multi_attribute_single_adapter_continued_sft/mistral_extractiveness_then_topic/model_extractiveness_and_topic_results_extractiveness_and_topic.pkl', '/scratch/tathagato/naacl/joint_mult_attribute_dpo/mistral_length_and_topic/model_length_and_topic_results_length_and_topic.pkl', '/scratch/tathagato/naacl/single_attribute_dpo/mistral_topic/results.pkl', '/scratch/tathagato/naacl/zero_shot/mistral_length_and_topic.pkl', '/scratch/tathagato/naacl/joint_mult_attribute_sft/mistral_length_and_topic/model_length_and_topic_results_length_and_topic.pkl', '/scratch/tathagato/naacl/joint_mult_attribute_dpo/mistral_extractiveness_and_topic/model_extractiveness_and_topic_results_extractiveness_and_topic.pkl', '/scratch/tathagato/naacl/zero_shot/mistral_topic_and_extractiveness.pkl', '/scratch/tathagato/naacl/zero_shot/mistral_topic.pkl', '/scratch/tathagato/naacl/joint_mult_attribute_sft/mistral_extractiveness_and_topic/model_extractiveness_and_topic_results_extr

In [5]:
attributes = ['topic']
arguments = [
    ["/scratch/tathagato/naacl/zero_shot/llama_topic.pkl", "llama"],
    ["/scratch/tathagato/naacl/zero_shot/mistral_topic.pkl", "mistral"],
    ["/scratch/tathagato/naacl/single_attribute_sft/llama_topic/results.pkl", "llama"],
    ["/scratch/tathagato/naacl/single_attribute_sft/mistral_topic/results.pkl", "mistral"],
    ["/scratch/tathagato/naacl/single_attribute_dpo/llama_topic/results.pkl", "llama"],
    ["/scratch/tathagato/naacl/single_attribute_dpo/mistral_topic/results.pkl", "mistral"]


]

def extract_floats(string):
    # This pattern matches numbers with optional negative sign,
    # decimal point, and exponent notation
    pattern = r'-?\d+(?:\.\d+)?(?:[eE][-+]?\d+)?'
    
    # Find all matches in the string
    matches = re.findall(pattern, string)
    
    # Convert matches to float
    floats = [float(match) for match in matches]
    
    return floats
def extract_words_around_then(filepath):
    # Check if "then" is in the filepath
    if "then" not in filepath:
        return ""
    
    # Regular expression to match the pattern
    pattern = r'_([^_]+)_then_([^_]+)_'
    
    # Search for the pattern in the filepath
    match = re.search(pattern, filepath)
    
    if match:
        # Extract the words before and after "then"
        word_before = match.group(1)
        word_after = match.group(2)
        return f"{word_before[0]}->{word_after[0]}"

def extract_meta_information(path):
    config = ""
    model = "Llama-3.1-Storm-8B"
    if "mistral" in path:
        model = "Mistral-7b-v3"
    method = "not sure"
    if "zero_shot" in path:
        method = "zero shot"

    if "adapter_fusion" in path:
        method = "adapter fusion"
        w1, w2 = extract_floats(path)[1], extract_floats(path)[2]
        config = f"{w1}, {w2}"

    elif "joint_mult_attribute" in path:
        method = "joint training"
    elif "multi_attribute_single_adapter_continued" in path:
        method = "single adapter continous"
        config = extract_words_around_then(path)
    elif "multi_attribute_multi_adapter" in path:
        method = "multi adapter"
        config = extract_words_around_then(path)
    
    objective = ""
    if "sft" in path:
        objective = "SFT"
    elif "dpo" in path:
        objective = "DPO"
    return model, method, objective, config



In [10]:
final_res = {}
for file, model in arguments:
    print(file, model)
    try:
        res = evaluate(file, model=model, attributes=attributes)
        print(res)
        final_res[file] = {"result": res, "model": model}
    except Exception as e:
        print(f"Error evaluating {file}: {e}")
        res = None
    


/scratch/tathagato/naacl/zero_shot/llama_topic.pkl llama
evaluating llama with attributes ['topic']
length of data is  266
/scratch/tathagato/naacl/zero_shot/llama_topic.pkl is being evaluated on topic


/home2/tathagato/miniconda3/envs/roy/lib/python3.9/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


--------------overall----------------
rouge1 : 0.297424114374473
rouge2 : 0.09756386448695305
rouge3 : 0.041503094260232735
rougeL : 0.19780125375875435
number : 266
-------------------------------------------------
{'topic': None}
/scratch/tathagato/naacl/zero_shot/mistral_topic.pkl mistral
evaluating mistral with attributes ['topic']
length of data is  266
/scratch/tathagato/naacl/zero_shot/mistral_topic.pkl is being evaluated on topic
--------------overall----------------
rouge1 : 0.16429442494439167
rouge2 : 0.058629450545901546
rouge3 : 0.027004938168936524
rougeL : 0.11200006088617187
number : 266
-------------------------------------------------
{'topic': None}
/scratch/tathagato/naacl/single_attribute_sft/llama_topic/results.pkl llama
evaluating llama with attributes ['topic']
length of data is  266
/scratch/tathagato/naacl/single_attribute_sft/llama_topic/results.pkl is being evaluated on topic
--------------overall----------------
rouge1 : 0.283551122023257
rouge2 : 0.0942235

In [9]:
for key in final_res:
    print(final_res[key])

{'result': {'topic': None}, 'model': 'llama'}
{'result': {'topic': None}, 'model': 'mistral'}
{'result': {'topic': None}, 'model': 'llama'}
{'result': {'topic': None}, 'model': 'mistral'}
{'result': {'topic': None}, 'model': 'llama'}
{'result': {'topic': None}, 'model': 'mistral'}


In [7]:
for file in final_res:
    print(file)
    model = final_res[file]["model"]
    print(model)
    display_length(final_res[file]["result"])
    print("")
    print("----------------------------------------------")

/scratch/tathagato/naacl/zero_shot/llama_topic.pkl
llama


NameError: name 'display_length' is not defined